# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
df_school = pd.read_csv(school_data_to_load)
df_student = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
df_complete = pd.merge(df_student, df_school, how="left", on=["school_name", "school_name"])

In [2]:
df_complete.head(2)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
total_schools = df_complete["School ID"].nunique()
total_students = df_complete["Student ID"].nunique()
total_students = "{0:,.0f}".format(total_students)

budget = []
for i in df_complete["budget"]:
    if i not in budget:
        budget.append(i)
total_budget = 0
for i in budget:
    total_budget += i
total_budget = "${0:,.2f}".format(total_budget)

avr_math_score = round(df_complete["math_score"].mean(),6)
avr_read_score = round(df_complete["reading_score"].mean(),6)

count_total = 0
count_passing_math = 0
count_passing_read = 0
count_passing_overall = 0
for i,j in zip(df_complete["math_score"],df_complete["reading_score"]):
    count_total += 1
    if i>=70:
        count_passing_math +=1
    if j>=70:
        count_passing_read += 1

math_passing_pct = round((count_passing_math/count_total) * 100,6)
read_passing_pct = round((count_passing_read/count_total) * 100,6)
wrong_overall_passing_pct = round((avr_read_score+avr_math_score)/2,6) # Notice that although this is wrong I still calculate it this way because you guys asked for this value. The correct way is given below
correct_overall_passing_pct = round(((count_passing_math + count_passing_read) / (2 * count_total)) * 100,6)

lst = [[total_schools, total_students, total_budget, avr_math_score, avr_read_score, math_passing_pct, 
        read_passing_pct, wrong_overall_passing_pct, correct_overall_passing_pct]]  
df_output = pd.DataFrame(lst, columns =['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 
                                        'Average Reading Score', '% Passing Math', '% Passing Reading', 
                                        '% Wrong Overall Passing Rate(Given in Ex Solution)', '% Correct Overall Passing Rate']) 
df_output

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Wrong Overall Passing Rate(Given in Ex Solution),% Correct Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
# Using GroupBy in order to separate the data into fields according to "school_name" values
grouped_df_complete = df_complete.groupby(['school_name'])

df_school_type = grouped_df_complete["type"].describe()
df_school_type= df_school_type [["top"]]
df_school_type = df_school_type.rename(columns = {"top":"School Type"})

df_total_students = grouped_df_complete[["Student ID"]].count()
df_total_students = df_total_students.rename(columns = {"Student ID":"Total Students"})
df_output2 = pd.merge(df_school_type,df_total_students, on="school_name")

df_budget = grouped_df_complete["budget"].describe()
df_budget = df_budget[["mean"]].rename(columns = {"mean":"Total School Budget"})
df_output2 = pd.merge(df_output2,df_budget, on="school_name")

df_output2["Per Student Budget"] = df_budget["Total School Budget"]/df_total_students["Total Students"]
df_output2["Total School Budget"] = df_output2["Total School Budget"].map("${:,.2f}".format)
df_output2["Per Student Budget"] = df_output2["Per Student Budget"].map("${:,.2f}".format)

df_avr_math_score = grouped_df_complete[["math_score"]].mean().rename(columns = {"math_score":"Average Math Score"})
df_output2 = pd.merge(df_output2,df_avr_math_score, on="school_name")

df_avr_read_score = grouped_df_complete[["reading_score"]].mean().rename(columns = {"reading_score":"Average Reading Score"})
df_output2 = pd.merge(df_output2,df_avr_read_score, on="school_name")

df_complete_pass_math = df_complete.loc[(df_complete["math_score"] >= 70)]
grouped_df_complete_pass_math = df_complete_pass_math.groupby(['school_name'])
df_students_pass_math = grouped_df_complete_pass_math[["math_score"]].count()
df_output2["% Passing Math"] = (df_students_pass_math["math_score"]/df_total_students["Total Students"]) * 100

df_complete_pass_read = df_complete.loc[(df_complete["reading_score"] >= 70)]
grouped_df_complete_pass_read = df_complete_pass_read.groupby(['school_name'])
df_students_pass_read = grouped_df_complete_pass_read[["reading_score"]].count()
df_output2["% Passing Reading"] = (df_students_pass_read["reading_score"]/df_total_students["Total Students"]) * 100

df_output2["% Overall Passing Rate"] = (df_output2["% Passing Math"] + df_output2["% Passing Reading"]) / 2
df_output2

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
df_top_performance = df_output2.sort_values("% Overall Passing Rate", ascending=False)
df_top_performance.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
df_bottom_performance = df_output2.sort_values("% Overall Passing Rate", ascending=True)
df_bottom_performance.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
df_9th_math = df_complete.loc[(df_complete["grade"] == "9th")]
grouped_df_9th_math = df_9th_math.groupby(['school_name'])
df_output3 = grouped_df_9th_math[["math_score"]].mean().rename(columns = {"math_score":"9th"})

df_10th_math = df_complete.loc[(df_complete["grade"] == "10th")]
grouped_df_10th_math = df_10th_math.groupby(['school_name'])
df_output3 = pd.merge(df_output3, grouped_df_10th_math[["math_score"]].mean().rename(columns = {"math_score":"10th"}), on="school_name")

df_11th_math = df_complete.loc[(df_complete["grade"] == "11th")]
grouped_df_11th_math = df_11th_math.groupby(['school_name'])
df_output3 = pd.merge(df_output3, grouped_df_11th_math[["math_score"]].mean().rename(columns = {"math_score":"11th"}), on="school_name")

df_12th_math = df_complete.loc[(df_complete["grade"] == "12th")]
grouped_df_12th_math = df_12th_math.groupby(['school_name'])
df_output3 = pd.merge(df_output3, grouped_df_12th_math[["math_score"]].mean().rename(columns = {"math_score":"12th"}), on="school_name")
df_output3

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
df_9th_read = df_complete.loc[(df_complete["grade"] == "9th")]
grouped_df_9th_read = df_9th_read.groupby(['school_name'])
df_output4 = grouped_df_9th_read[["reading_score"]].mean().rename(columns = {"reading_score":"9th"})

df_10th_read = df_complete.loc[(df_complete["grade"] == "10th")]
grouped_df_10th_read = df_10th_read.groupby(['school_name'])
df_output4 = pd.merge(df_output4, grouped_df_10th_read[["reading_score"]].mean().rename(columns = {"reading_score":"10th"}), on="school_name")

df_11th_read = df_complete.loc[(df_complete["grade"] == "11th")]
grouped_df_11th_read = df_11th_read.groupby(['school_name'])
df_output4 = pd.merge(df_output4, grouped_df_11th_read[["reading_score"]].mean().rename(columns = {"reading_score":"11th"}), on="school_name")

df_12th_read = df_complete.loc[(df_complete["grade"] == "12th")]
grouped_df_12th_read = df_12th_read.groupby(['school_name'])
df_output4 = pd.merge(df_output4, grouped_df_12th_read[["reading_score"]].mean().rename(columns = {"reading_score":"12th"}), on="school_name")
df_output4

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [10]:
df_output5 = df_output2
df_output5["Per Student Budget"] = df_output5["Per Student Budget"].replace({'\$':''}, regex = True)
df_output5["Per Student Budget"] = df_output5["Per Student Budget"].astype(float)

In [11]:
df_output5["Spending Ranges (Per Student)"] = pd.cut(df_output5["Per Student Budget"], bins=spending_bins, labels = group_names)
df_output5 = df_output5.reset_index()

In [12]:
grouped_df_output5 = df_output5.groupby(['Spending Ranges (Per Student)'])

In [13]:
df_output5 = grouped_df_output5.mean().head()
df_output5 = df_output5.drop(["Total Students","Per Student Budget"], axis =1)
df_output5.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [14]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names_2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [15]:
df_output6 = df_output2
df_output6["School Size"] = pd.cut(df_output6["Total Students"], bins=size_bins, labels = group_names_2)
df_output5 = df_output5.reset_index()

In [16]:
grouped_df_output6 = df_output6.groupby(['School Size'])

In [17]:
df_output6 = grouped_df_output6.mean().head()
df_output6 = df_output6.drop(["Total Students","Per Student Budget"], axis =1)
df_output6.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [18]:
df_output7 = df_output2
df_output7 = df_output7.reset_index()

In [19]:
grouped_df_output7 = df_output7.groupby(['School Type'])

In [20]:
df_output7 = grouped_df_output7.mean().head()
df_output7 = df_output7.drop(["Total Students","Per Student Budget"], axis =1)
df_output7.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
